## Problem 2

In [55]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
from scipy import stats
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.stats.sandwich_covariance import weights_bartlett

# ------------------------------------------------------------
# Load data
# ------------------------------------------------------------
df = pd.read_excel("ccapm.xlsx", header=None,
                   names=["cratio", "rrate", "e"])

cr = df["cratio"].to_numpy()
R = df["rrate"].to_numpy()
e = df["e"].to_numpy()

# We need R_{t+1} etc. with instruments at time t,
# so we drop the first observation after constructing lags.
cr_fwd = cr[1:]      # c_{t+1}/c_t
R_fwd  = R[1:]       # R_{t+1}
e_fwd  = e[1:]       # e_{t+1}

cr_lag = cr[:-1]     # c_t/c_{t-1}
R_lag  = R[:-1]      # R_t
e_lag  = e[:-1]      # e_t

N = len(cr_fwd)      # effective observations


# ------------------------------------------------------------
# GMM model class
# ------------------------------------------------------------
class CCAPMGMM(GMM):
    """
    Non-linear GMM for the CCAPM Euler equation

        E_t[ beta (c_{t+1}/c_t)^(-gamma) R_{t+1} - 1 ] = 0

    with 4 instruments: 1, (c_t/c_{t-1}), R_t, e_t.
    """
    def __init__(self, cr, R, e):
        cr_fwd = cr[1:]
        R_fwd  = R[1:]
        e_fwd  = e[1:]
        cr_lag = cr[:-1]
        R_lag  = R[:-1]
        e_lag  = e[:-1]

        # pack data; first three columns = "current" variables,
        # last three columns = instruments
        data = np.column_stack([cr_fwd, R_fwd, e_fwd,
                                cr_lag, R_lag, e_lag])
        endog = np.zeros(len(cr_fwd))       # not used
        exog  = data[:, :3]                 # cr_{t+1}/c_t, R_{t+1}, e_{t+1}
        instr = data[:, 3:]                 # c_t/c_{t-1}, R_t, e_t

        # 4 moments: 1, c_t/c_{t-1}, R_t, e_t
        super().__init__(endog, exog, instr, nmoms=4)
    
    def momcond(self, params):
        beta, gamma = params

        cr_fwd = self.exog[:, 0]
        R_fwd  = self.exog[:, 1]
        e_fwd  = self.exog[:, 2]

        cr_lag = self.instrument[:, 0]
        R_lag  = self.instrument[:, 1]
        e_lag  = self.instrument[:, 2]

        m1 = beta * (cr_fwd ** (-gamma)) * R_fwd - 1.0

        g = np.column_stack([
            m1,
            m1 * cr_lag,
            m1 * R_lag,
            m1 * e_lag
        ])
        return g


theta0 = np.array([1.0, 1.0])  # starting values beta=1, gamma=1


# ------------------------------------------------------------
# Part (a): statsmodels two–step GMM 
# ------------------------------------------------------------
model_a = CCAPMGMM(cr, R, e)

# maxiter=2 -> standard two–step GMM in statsmodels
res_a = model_a.fit(start_params=theta0,
                    maxiter=2,
                    optim_method="bfgs",   # 默认也是 bfgs，这里写出来更清楚
                    weights_method="cov")  # HC0

beta_a, gamma_a = res_a.params
se_beta_a, se_gamma_a = res_a.bse

print("Part (a) -- statsmodels two–step GMM (HC0)")
print("beta_hat  =", beta_a, "  se(beta)  =", se_beta_a)
print("gamma_hat =", gamma_a, "  se(gamma)=", se_gamma_a)
print("J-stat    =", res_a.jval)


# ------------------------------------------------------------
# Part (b): statsmodels GMM + HAC (Newey–West, maxlag=5)
# ------------------------------------------------------------
model_b = CCAPMGMM(cr, R, e)

res_b = model_b.fit(
    start_params=theta0,
    maxiter=100,                
    weights_method="hac",
    wargs={"maxlag": 5, "kernel": weights_bartlett},
    optim_method="bfgs"
)

beta_b, gamma_b = res_b.params
se_beta_b, se_gamma_b = res_b.bse

print("\nPart (b) -- HAC GMM (Newey–West, maxlag=5)")
print("beta_hat  =", beta_b,  "  se(beta)  =", se_beta_b)
print("gamma_hat =", gamma_b, "  se(gamma)=", se_gamma_b)
print("J-stat    =", J_b)


# ------------------------------------------------------------
# Part (c): test H0: beta = 0.98 using HAC s.e. (from res_b)
# ------------------------------------------------------------
beta0 = 0.98
t_stat = (beta_b - beta0) / se_beta_b
p_value = 2 * stats.norm.sf(abs(t_stat))

print("\nPart (c) -- test H0: beta = 0.98 (using HAC s.e.)")
print("t-stat =", t_stat, "  p-value =", p_value)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005432
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Part (a) -- statsmodels two–step GMM (HC0)
beta_hat  = 0.9976781570506459   se(beta)  = 0.004330625203063674
gamma_hat = 0.7285765245094432   se(gamma)= 1.771092191687716
J-stat    = 1.2873602203880312
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.008025
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.007271
         Iterations: 4
         Functi

## Problem 4

In [53]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# ---------------------------------------------------------
# Load and clean, create FTE 
# ---------------------------------------------------------
df = pd.read_csv("minwage.txt", sep="\t")
df = df.replace('.', pd.NA)

for col in ['empft','emppt','nmgrs','wagest',
            'empft2','emppt2','nmgrs2','wagest2']:
    df[col] = pd.to_numeric(df[col])
    

cols_required = [
    'wagest',  'wagest2',      # wages before/after
    'empft',   'emppt', 'nmgrs',   # employment before
    'empft2',  'emppt2', 'nmgrs2'  # employment after
]

df_clean = df.dropna(subset=cols_required).copy()
df_clean = df_clean.reset_index(drop=True)

df_clean['fte']  = df_clean['empft']  + df_clean['nmgrs']  + 0.5 * df_clean['emppt']
df_clean['fte2'] = df_clean['empft2'] + df_clean['nmgrs2'] + 0.5 * df_clean['emppt2']

# ---------------------------------------------------------
# Build long panel: one row per restaurant-period (Before/After)
# ---------------------------------------------------------
n = df_clean.shape[0]

before = pd.DataFrame({
    'id':   np.arange(n),
    'state': df_clean['state'],       # 0=PA, 1=NJ
    'chain': df_clean['chain'],
    'own':   df_clean['own'],
    'time':  0,                       # Before
    'fte':   df_clean['fte']
})
after = pd.DataFrame({
    'id':   np.arange(n),
    'state': df_clean['state'],
    'chain': df_clean['chain'],
    'own':   df_clean['own'],
    'time':  1,                       # After
    'fte':   df_clean['fte2']
})

panel = pd.concat([before, after], ignore_index=True)

panel['D']      = (panel['state'] == 1).astype(int)  # 1=NJ
panel['Time']   = panel['time']                     # 1=After
panel['D_Time'] = panel['D'] * panel['Time']

### (a)

In [44]:
df_clean["d_fte"] = df_clean["fte2"] - df_clean["fte"]

group_means = df_clean.groupby("state")[["fte", "fte2", "d_fte"]].mean()

fte_PA_before = group_means.loc[0, "fte"]
fte_PA_after  = group_means.loc[0, "fte2"]
fte_NJ_before = group_means.loc[1, "fte"]
fte_NJ_after  = group_means.loc[1, "fte2"]

# Difference-in-differences
dd = (fte_NJ_after - fte_NJ_before) - (fte_PA_after - fte_PA_before)

print("PA before =", fte_PA_before)
print("PA after  =", fte_PA_after)
print("NJ before =", fte_NJ_before)
print("NJ after  =", fte_NJ_after)
print("DID (NJ - PA, after - before) =", dd)

PA before = 23.704545454545453
PA after  = 21.825757575757574
NJ before = 20.678245614035088
NJ after  = 21.076315789473686
DID (NJ - PA, after - before) = 2.2768580542264765


### (b)

In [45]:
Xb = sm.add_constant(panel[['D', 'Time', 'D_Time']])
mod_b = sm.OLS(panel['fte'], Xb).fit(
    cov_type='cluster', cov_kwds={'groups': panel['id']}
)
print(mod_b.summary())

theta_hat = mod_b.params['D_Time']
se_theta  = mod_b.bse['D_Time']


                            OLS Regression Results                            
Dep. Variable:                    fte   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.276
Date:                Thu, 04 Dec 2025   Prob (F-statistic):              0.283
Time:                        13:57:08   Log-Likelihood:                -2561.8
No. Observations:                 702   AIC:                             5132.
Df Residuals:                     698   BIC:                             5150.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.7045      1.514     15.655      0.0

### (c)

In [46]:
chain_dums = pd.get_dummies(panel['chain'], prefix='chain', drop_first=True)
Xc = pd.concat([panel[['D', 'Time', 'D_Time', 'own']], chain_dums], axis=1)
Xc = sm.add_constant(Xc)

mod_c = sm.OLS(panel['fte'], Xc).fit(
    cov_type='cluster', cov_kwds={'groups': panel['id']}
)
print(mod_c.summary())

theta_hat_c = mod_c.params['D_Time']
se_theta_c  = mod_c.bse['D_Time']

                            OLS Regression Results                            
Dep. Variable:                    fte   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     37.61
Date:                Thu, 04 Dec 2025   Prob (F-statistic):           3.64e-39
Time:                        13:57:10   Log-Likelihood:                -2478.3
No. Observations:                 702   AIC:                             4973.
Df Residuals:                     694   BIC:                             5009.
Df Model:                           7                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.3896      1.521     17.350      0.0

### (d)

In [47]:
NEW_MIN = 5.05
df_cs = df_clean.copy()
df_cs['d_fte'] = df_cs['fte2'] - df_cs['fte']
df_cs['gap']   = np.maximum(0, (NEW_MIN - df_cs['wagest'])/df_cs['wagest'])
df_cs.loc[df_cs['state'] == 0, 'gap'] = 0


Xd = sm.add_constant(df_cs['gap'])
mod_d = sm.OLS(df_cs['d_fte'], Xd).fit()
print(mod_d.summary())

beta_gap = mod_d.params['gap']
se_gap   = mod_d.bse['gap']

mean_gap_NJ = df_cs.loc[df_cs['state'] == 1, 'gap'].mean()
implied_effect = mean_gap_NJ * beta_gap
implied_effect

                            OLS Regression Results                            
Dep. Variable:                  d_fte   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     7.839
Date:                Thu, 04 Dec 2025   Prob (F-statistic):            0.00540
Time:                        13:57:12   Log-Likelihood:                -1254.9
No. Observations:                 351   AIC:                             2514.
Df Residuals:                     349   BIC:                             2522.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4777      0.694     -2.130      0.0

1.7829285215831532

In [56]:
mean_gap_NJ

0.10456091708582543

### (e)

In [48]:
Xe = pd.concat([
    df_cs[['gap', 'state', 'own']],
    pd.get_dummies(df_cs['chain'], prefix='chain', drop_first=True)
], axis=1)
Xe = sm.add_constant(Xe)

mod_e = sm.OLS(df_cs['d_fte'], Xe).fit()
print(mod_e.summary())

beta_gap_e   = mod_e.params['gap']
se_gap_e     = mod_e.bse['gap']
beta_state_e = mod_e.params['state']
se_state_e   = mod_e.bse['state']

                            OLS Regression Results                            
Dep. Variable:                  d_fte   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.797
Date:                Thu, 04 Dec 2025   Prob (F-statistic):             0.0988
Time:                        13:57:16   Log-Likelihood:                -1253.4
No. Observations:                 351   AIC:                             2521.
Df Residuals:                     344   BIC:                             2548.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6635      1.211     -1.374      0.1